# Config Session

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

import platform
print(platform.platform())
import torch
print(torch.__version__)
!nvcc --version

Mon Nov 13 15:23:36 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.13                 Driver Version: 537.13       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090      WDDM  | 00000000:01:00.0  On |                  Off |
|  0%   43C    P8              16W / 500W |   7793MiB / 24564MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

C:\Users\rvc_sw_mss1_common\Desktop\test\venv\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\rvc_sw_mss1_common\Desktop\test\venv\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\rvc_sw_mss1_common\Desktop\test\venv\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


2.0.1+cu118
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:41:10_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 136.5 gigabytes of available RAM

You are using a high-RAM runtime!


# SETUP Modules

In [3]:
######################
#  MODEL & TOKENIZER #
######################
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [4]:
####################
# DOCUMENT LOADERS #
####################
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PyPDFLoader as PyPDFLoader
from langchain.document_loaders import PyMuPDFLoader as PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter

In [5]:
####################
#  VECTOR STORES   #
####################
from langchain.vectorstores import FAISS

# LLM Model Initialization

In [6]:
####################
#  EMBEDDING       #
####################
embedding_path = "C:\\Users\\rvc_sw_mss1_common\\Desktop\\test\\mcal-guru\\model\\bge-large-en-v1.5"
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name=embedding_path)

In [7]:
######################
#  MODEL & TOKENIZER #
#   INITIALIZATION   #
######################
model_dir = "C:\\Users\\rvc_sw_mss1_common\\Desktop\\test\\mcal-guru\\model\\OpenOrca-Platypus2-13B-GPTQ"

from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
from transformers import AutoModelForCausalLM

quantize_config = BaseQuantizeConfig(
    bits=4,  # quantize model to 4-bit
    #damp_percent= 0.01,
    desc_act= False,
    group_size=128,  # it is recommended to set the value to 128
)

#model = AutoGPTQForCausalLM.from_quantized(pretrained_model_name_or_path=model_dir,
                                            #device="cuda:0")

model = AutoGPTQForCausalLM.from_quantized(
    model_dir,
    # strict=False,
    use_triton=False,
    map_device='auto',
    use_safetensors=True,
    quantize_config=quantize_config,
    model_basename='model'
)

tokenizer = AutoTokenizer.from_pretrained(model_dir, map_device='auto')

# 3m40s when using A100

  0%|          | 0/1483 [00:00<?, ?w/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

skip module injection for FusedLlamaMLPForQuantizedModel not support integrate without triton yet.


# Handle Documents

In [8]:
######################
#  FILE HANDLER      #
######################
import glob
import pickle
from langchain.storage import InMemoryStore

In [9]:
###############################################################################################
#  PROCESS: # Split the documents into chunks
###############################################################################################

#text_splitter = CharacterTextSplitter(
#    separator = "\n",
#    chunk_size = 1000,
#    chunk_overlap  = 0,
#    length_function = len,
#  )
#
#child_text_splitter = CharacterTextSplitter(
#    separator = "\n",
#    chunk_size = 2000,
#    chunk_overlap  = 0,
#    length_function = len,
#  )
#
#parent_text_splitter = CharacterTextSplitter(
#    separator = "\n",
#    chunk_size = 500,
#    chunk_overlap  = 0,
#    length_function = len,
#  )
#
## The storage layer for parent docs
#store = InMemoryStore()

In [10]:
%%time
###############################################################################################
#  PROCESS: Extract text from multiple PDF documents and save as .pickle file
###############################################################################################

# Read PDF file
#directory_path = 'C:\\Users\\rvc_sw_mss1_common\\Desktop\\test\\mcal-guru\\01_Middle_Data\\02_Data_Input\\pdf\\'
#vector_db_path = 'C:\\Users\\rvc_sw_mss1_common\\Desktop\\test\\mcal-guru\\01_Middle_Data\\02_Data_Input\\vector_db\\'
#file_paths = glob.glob(directory_path + "/*")
#docs = []
#number = 0
#for file_path in file_paths:
#  if number%2 == 0:
#    docs = []
#  loader = PyPDFLoader(file_path)
#  docs.extend(loader.load())
#  texts = text_splitter.split_documents(docs)
#  if number%2 == 1:
#    # save the docs variable to a file
#    with open(vector_db_path + '\\docs_' + str((number+1)/2) + '.pickle', 'wb') as f:
#        pickle.dump(texts, f)
#  number += 1

CPU times: total: 0 ns
Wall time: 0 ns


In [11]:
#%%time
################################################################################################
##  PROCESS: Load documents from .pickle file
################################################################################################
#
## use glob to get all the file paths in the directory
#file_paths = glob.glob(vector_db_path + "/*")
#
#documents = []
#number = 0
#for file_path in file_paths:
#  with open(file_path, 'rb') as f:
#    text = pickle.load(f)
#  documents.extend(text)
#  number += 1
#print(len(documents))
#print(number)

In [12]:
%%time
###############################################################################################
#  PROCESS: # Split the documents into chunks
###############################################################################################
#db = FAISS.from_documents(document_1, embeddings
db = FAISS.from_texts(["This is just a demo texts"], embeddings)

# 29.5s when using A100

CPU times: total: 1.11 s
Wall time: 1.1 s


In [13]:
indexed_data_path = 'C:\\Users\\rvc_sw_mss1_common\\Desktop\\test\\mcal-guru\\01_Middle_Data\\02_Data_Input\\vector_db//extracted_data.pkl'
with open(indexed_data_path, 'rb') as f:
    indexed_data = pickle.load(f)

In [16]:
i = 0
sub_data = indexed_data[30467:30472:2]
for item in sub_data:
    page_content = [item['page_content']]
    metadata = [item['metadata']]
    print(f"\n\n[DEBUG] page_content: {page_content}")
    print(f"[DEBUG] metadata: {metadata}")



[DEBUG] page_content: ['Test program:  In the "Gentool Unit Test" phase of MCAL development, the "Test program .cs .fakes" refers to the files used for writing and conducting unit tests for the Generation Tool (Gentool). These files are an essential part of the testing process to ensure the correctness and reliability of the Generation Tool\'s functionalities.\n\n1. Test Program (.cs):\nThe ".cs" files in the "Gentool Unit Test" phase are C# source code files that contain the unit test cases. These test cases are designed to verify the individual units or components of the Generation Tool in isolation. The test program includes test methods that exercise various functionalities of the Gentool, simulating different scenarios and inputs. Each test method focuses on testing a specific feature or behavior of the tool. The test program interacts with the Generation Tool through its API or interfaces, checking the expected outputs against the actual results.\n\n2. Fakes Framework (.fakes):

In [12]:
%%time
# Copy indexed_data to create the vector database
for item in indexed_data:
    page_content = [item['page_content']]
    metadata = [item['metadata']]
    db.add_texts(texts=page_content, metadatas=metadata, embdeding=embeddings)

CPU times: total: 7min 7s
Wall time: 7min 5s


In [13]:
res_docs = db.similarity_search_with_score(query="is VIN included in MCAL Software", k=2)
for doc in res_docs:
    print("\n\n####################################")
    print(f"[DEBUG] type of doc: {type(doc)}")
    print(f"[DEBUG] sim score: {doc[1]}")
    print(doc)
    



####################################
[DEBUG] type of doc: <class 'tuple'>
[DEBUG] sim score: 0.5729751586914062
(Document(page_content='Configuration files:  In the "Driver Integrated Test" phase of MCAL development, the "Configuration files" refer to the "*.arxml" files that contain the AUTOSAR (AUTomotive Open System ARchitecture) XML-based configuration data. These files play a crucial role in configuring and initializing the MCAL drivers and other software components of the automotive application. The "Configuration files" typically include the following key information: 1. Software Component Configuration: The "*.arxml" files define the configuration parameters for each MCAL driver and other software components used in the application. These parameters include various settings, constants, and options that tailor the behavior of the drivers to meet specific requirements. 2. Hardware Configuration: The files may also contain information related to the hardware architecture and pla

In [19]:
res_docs = db.similarity_search_with_score(query="Hello, could you tell me something that could possibly make me fun", k=2)
for doc in res_docs:
    print("\n\n####################################")
    print(doc)



####################################
(Document(page_content='This is just a demo texts'), 0.85889506)


####################################
(Document(page_content='0 Other setting is prohibited :  0 Other setting is prohibited ', metadata={'source': '3. Pin Assignment ', 'page': '4'}), 0.95692384)


In [21]:
res_docs = db.similarity_search_with_score(query="Give me the source code of the Snake game", k=2)
for doc in res_docs:
    print("\n\n####################################")
    print(doc)



####################################
(Document(page_content="1.2.3 Debug and Trace :  Item Description  'HEXJD QG7U•-7$*LQWHUIDFH   6XSSRUWVS GLQ H G- L7$FD* WH G6:' DQG/3'  $GGLWLP RX QO DWLOSS- O7$*SLH Q[H GR UW •7KHFURVVWULJIR JHOO URZL IX QQ FJ WL, R3 QV E HWZHH  &RUW$H[&38Q  &RUW5H[&38Q  9LVLRQ'63&(9$63\x8c  ,03  &RUH6LJKW  ,&80 ; 3$6\\3 VWHP •7UDFLQJIXQFWLRQ  *ESVRU*ESV+6673 RXWSX  *ESVRU*ESV+6673ODQ RH XWS X  *ESRUV*ESV(75RXWS XWEDQG  *ESVRU*ESV(75ROD XQ WH S XWE  .E \\WHHPEHGGHG$W UD FH),)2IR  .E \\WHHPHE)HG, G) H2 GI5WR UDU&RFUWH[  .E \\WHHPEHGGHGW UDFH),)2IR  .E \\WHHPEHGGHGWUDF63 H)\x8c ,) 2IR  6\\VWHP7UDFH0DFUR FHOO,630 •6HFXUH GHEXJ  'HEXJJLQJLVGLVDEOHGLIVHFXU HQDEOHGZLWKVHFXUHGHEXJJLQJDXWK /VW3'DQGVIRU/RZ3LQ'H EXJZKLFKLVXVHGIRU,&8 $YDLODEt Oab Hle fro Em e Dithe Qr 5 GGb Zps p Ler Glan WKe or 2 .5G Vbp Hs p Oer Hlan Fe $YDLODEt Oab Hle fro Em e Dithe Qr 5 GGb Zps p Ler Glan WKe or 2 .5G Vbp Hs p Oer Hlan Fe $YDLODEt Oab Hle fro Em e Dithe Qr 1 G6G Zbps Lpe Gr lan WKe or

# **Set Up QA Chain Tool**

## Init Chain

In [15]:
from langchain.chains import ConversationalRetrievalChain
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ParentDocumentRetriever


In [18]:
################################
#  QA Retriver initialization  #
################################



retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})
qa = ConversationalRetrievalChain.from_llm(llm=llm_local, retriever=retriever)

In [19]:
chat_history = []
def qa_run(query):
    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result["answer"]))
    return result["answer"]

In [20]:
%%time
qa_run("What is ADC_Init?")

CPU times: total: 13.8 s
Wall time: 9 s


' ADC_Init is a function provided by the ADC driver which initializes the internal Analogue Digital Converter unit(s) of the microcontroller. This function sets up the necessary parameters and configurations for the ADC to operate correctly. It is typically called before any other ADC functions are utilized.'

In [21]:
%%time
query = "What is SRS_Adc_12307 in AUTOSAR?"
result = qa({"question": query, "chat_history": chat_history})
chat_history.append((query, result["answer"]))
result["answer"]

CPU times: total: 29.6 s
Wall time: 16.7 s


' ADC_Init is responsible for initializing and configuring the internal Analogue Digital Converter unit(s) within the microcontroller. This includes starting and stopping conversions, enabling and disabling the trigger source, controlling the notification function, and checking the status and results of conversions. In summary, ADC_Init sets up the analog-to-digital converter by preparing its components and establishing communication between them.'

In [ ]:
## MultiQueryRetriever

In [33]:
################################
#  QA Retriver initialization  #
################################


retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})


In [34]:
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers").setLevel(logging.INFO)

In [38]:
rel_docs = retriever.get_relevant_documents(query="What is SRS_Adc_12307")
print(rel_docs)

[Document(page_content='Requirements on ADC Driver  \nAUTOSAR CP R21-11    \n \n \n12 of 22 Document ID 111: AUTOSAR_SRS_ADCDriver  \n   \n5 Requirements Tracing  \n \n \nRequirement  Description  Satisfied by  \nRS_BRF_01184  AUTOSAR shall support \ndifferent methods of \ndegradation  SRS_ADC_12826, SRS_ADC_12827, \nSRS_ADC_12828, SRS_ADC_12829  \nRS_BRF_01448  AUTOSAR services shall \nsupport mode and state \nmanagement  SRS_ADC_12826, SRS_ADC_12827, SRS_ADC_12828  \nRS_BRF_01872  AUTOSAR \nmicrocontroller \nabstraction shall provide \nmapping of I/O signals to \nanalog/digital converter \nports  SRS_Adc_12280, SRS_Adc_12283, SRS_Adc_12288, \nSRS_Adc_12291, SRS_Adc_12292, SRS_Adc_12307, \nSRS_Adc_12317, SRS_Adc_12318, SRS_Adc_12364, \nSRS_Adc_12447, SRS_Adc_12802, SRS_Adc_12817, \nSRS_Adc_12818, SRS_Adc_12819, SRS_Adc_12820, \nSRS_Adc_12821, SRS_Adc_12822, SRS_Adc_12823, \nSRS_Adc_12824, SRS_Adc_12825  \nRS_BRF_01952  AUTOSAR IO Hardware \nAbstraction sh all support \nstandardized mo

In [29]:
%%time
query = "What is SRS_Adc_12307 in AUTOSAR?"
qa_run(query)

CPU times: total: 1min 20s
Wall time: 49.4 s


" The provided text does not give enough information about SRS_Adc_12307 in AUTOSAR. It only mentions that SRS_Adc_12307 is related to ASR documentation which deals with automotive software robustness. More context or information about this specific term is needed to provide a clear explanation.\n\nOriginal Answer: SRS_Adc_12307 in AUTOSAR refers to an Automotive Software Robustness Specification (ASR) document that focuses on the design and implementation of automotive software systems. However, without more context or information about this specific term, it's difficult to provide a detailed answer. If you could provide additional details or background, I would be happy to help clarify what SRS_Adc_12307 means in relation to AUTOSAR. Helpful Answer: The provided text does not give enough information about SRS_Adc_12307 in AUTOSAR. It only mentions that SRS_Adc_12307 is related to ASR documentation which deals with automotive software robustness. More context or information about this